In [ ]:
import torch
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

In [ ]:
# https://pytorch.org/tutorials/beginner/introyt/captumyt.html

In [ ]:
# https://captum.ai/tutorials/Bert_SQUAD_Interpret
# https://github.com/pytorch/captum/issues/373

In [ ]:
# read predictions
predicitons = pd.read_csv("/content/drive/My Drive/LIN371/predictions.csv")
predicitons.head()

,Unnamed: 0,text,label,new_label,prediction
0,0,Those pussy lips need more cleaning with my to...,1,explicit_source_has_explicit_words,1
1,1,"I have choices for you. Choice seating, at that",1,explicit_source_no_explicit_words,1
2,2,I want to finish.,1,explicit_source_no_explicit_words,0
3,3,"Oh it Will, one way or an other 😉",1,explicit_source_no_explicit_words,1
4,4,"No need to thank me, thank you so much for sha...",1,explicit_source_no_explicit_words,0


In [ ]:
# get some examples where pred is 1

pred_1 = predicitons[predicitons['prediction'] == 1].sample(5, random_state=5)
texts = pred_1['text'].tolist()
texts

['Looks wet and ready',
 'Come here you cuddly little rascal',
 'Exactly what I need after a long day at work.',
 'man i love a Bush! u are stunning',
 'Mmmmm mmmm mmm\n']

Integrated Gradients is one of the feature attribution algorithms available in Captum. Integrated Gradients assigns an importance score to each input feature by approximating the integral of the gradients of the model’s output with respect to the inputs.

In [ ]:
import transformers
from transformers import BertTokenizer, BertForSequenceClassification
# replace <PATH-TO-SAVED-MODEL> with the real path of the saved model
model_path = '/content/drive/My Drive/LIN371/classification_only_model'

# set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# # load model
# model = BertForSequenceClassification.from_pretrained(model_path)
# model.to(device)
# model.eval()
# model.zero_grad()
# # load tokenizer
# tokenizer = BertTokenizer.from_pretrained(model_path)


model = BertForSequenceClassification.from_pretrained('/content/drive/My Drive/LIN371/bert-base-uncased-mlm-classifier')
tokenizer = BertTokenizer.from_pretrained('/content/drive/My Drive/LIN371/bert-base-uncased-mlm-classifier')
model.to(device)
model.eval()
model.zero_grad()

In [ ]:
def predict(inputs, token_type_ids=None, position_ids=None, attention_mask=None):
    output = model(
        inputs,
        token_type_ids=token_type_ids,
        position_ids=position_ids,
        attention_mask=attention_mask
    )
    return output.logits  # For sequence classification


In [ ]:
def class_forward_func(inputs, attention_mask=None):
    pred = predict(inputs,
                   attention_mask=attention_mask)
    class_1_logits = pred[:,1]
    return class_1_logits

In [ ]:
sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id # A token used for prepending to the concatenated question-text word sequence

In [ ]:
!pip install captum

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.3 MB/s eta 0:00:00


In [ ]:
from captum.attr import LayerConductance, LayerIntegratedGradients

In [ ]:
def integrated_grad(model, tokenizer, text):


    inputs = tokenizer(
    [text],
    return_tensors="pt",
    truncation=True,
    max_length=512,
    #padding='max_length',
)

    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]
    pred = predict(inputs=input_ids.to(device), attention_mask=attention_mask.to(device))

    lig = LayerIntegratedGradients(class_forward_func, model.bert.embeddings)

    pad_token_id = tokenizer.pad_token_id
    ref_input_ids = torch.full_like(input_ids, pad_token_id).to(device)
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)


    attributions, delta = lig.attribute(
        inputs=input_ids,
        baselines=ref_input_ids,  # Baseline, `[PAD]` token embeddings
        additional_forward_args=( attention_mask),
        return_convergence_delta=True
    )

    return attributions, delta


In [ ]:
def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    return attributions

In [ ]:
def summarize_integrated_grad(model, tokenizer, text):
  attributions, delta = integrated_grad(model, tokenizer, text)
  summarized_attributions = summarize_attributions(attributions)
  return summarized_attributions

In [ ]:
text0_attributions = summarize_integrated_grad(model, tokenizer, texts[0])
text1_attributions = summarize_integrated_grad(model, tokenizer, texts[1])
text2_attributions = summarize_integrated_grad(model, tokenizer, texts[2])
text3_attributions = summarize_integrated_grad(model, tokenizer, texts[3])
text4_attributions = summarize_integrated_grad(model, tokenizer, texts[4])

In [ ]:
# load the mlm model
mlm_model = BertForSequenceClassification.from_pretrained('/content/drive/My Drive/LIN371/bert-base-uncased-mlm-classifier')
mlm_tokenizer = BertTokenizer.from_pretrained('/content/drive/My Drive/LIN371/bert-base-uncased-mlm-classifier')
mlm_model.to(device)
mlm_model.eval()
mlm_model.zero_grad()


In [ ]:
mlm0_attributions = summarize_integrated_grad(model, tokenizer, texts[0])
mlm1_attributions = summarize_integrated_grad(model, tokenizer, texts[1])
mlm2_attributions = summarize_integrated_grad(model, tokenizer, texts[2])
mlm3_attributions = summarize_integrated_grad(model, tokenizer, texts[3])
mlm4_attributions = summarize_integrated_grad(model, tokenizer, texts[4])


In [ ]:
text0_tokens = tokenizer(texts[0], return_tensors="pt")
text0_tokens = tokenizer.convert_ids_to_tokens(text0_tokens['input_ids'][0].numpy())
text1_tokens = tokenizer(texts[1], return_tensors="pt")
text1_tokens = tokenizer.convert_ids_to_tokens(text1_tokens['input_ids'][0].numpy())
text2_tokens = tokenizer(texts[2], return_tensors="pt")
text2_tokens = tokenizer.convert_ids_to_tokens(text2_tokens['input_ids'][0].numpy())
text3_tokens = tokenizer(texts[3], return_tensors="pt")
text3_tokens = tokenizer.convert_ids_to_tokens(text3_tokens['input_ids'][0].numpy())
text4_tokens = tokenizer(texts[4], return_tensors="pt")
text4_tokens = tokenizer.convert_ids_to_tokens(text4_tokens['input_ids'][0].numpy())

In [ ]:
text0_df = pd.DataFrame({'token': [token for token in text0_tokens],
                         'int_grad': [score.item() for score in text0_attributions],
                         'mlm_int_grad': [score.item() for score in mlm0_attributions]})
text1_df = pd.DataFrame({'token': [token for token in text1_tokens],
                         'int_grad': [score.item() for score in text1_attributions],
                         'mlm_int_grad': [score.item() for score in mlm1_attributions]})
text2_df = pd.DataFrame({'token': [token for token in text2_tokens],
                         'int_grad': [score.item() for score in text2_attributions],
                         'mlm_int_grad': [score.item() for score in mlm2_attributions]})
text3_df = pd.DataFrame({'token': [token for token in text3_tokens],
                         'int_grad': [score.item() for score in text3_attributions],
                         'mlm_int_grad': [score.item() for score in mlm3_attributions]})
text4_df = pd.DataFrame({'token': [token for token in text4_tokens],
                         'int_grad': [score.item() for score in text4_attributions],
                         'mlm_int_grad': [score.item() for score in mlm4_attributions]})

In [ ]:
# update the df

update_text0

update_text1_df = pd.read_csv('/content/drive/My Drive/LIN371/example_sent/text1_df')
update_text1_df['int_grad'] = text1_df['int_grad']
update_text1_df['mlm_int_grad'] = text1_df['mlm_int_grad']
update_text1_df.to_csv('/content/drive/My Drive/LIN371/example_sent/text1_df.csv')

update_text2_df = pd.read_csv('/content/drive/My Drive/LIN371/example_sent/text2_df')
update_text2_df['int_grad'] = text2_df['int_grad']
update_text2_df['mlm_int_grad'] = text2_df['mlm_int_grad']
update_text2_df.to_csv('/content/drive/My Drive/LIN371/example_sent/text2_df.csv')

update_text3_df = pd.read_csv('/content/drive/My Drive/LIN371/example_sent/text3_df')
update_text3_df['int_grad'] = text3_df['int_grad']
update_text3_df['mlm_int_grad'] = text3_df['mlm_int_grad']
update_text3_df.to_csv('/content/drive/My Drive/LIN371/example_sent/text3_df.csv')

update_text4_df = pd.read_csv('/content/drive/My Drive/LIN371/example_sent/text4_df')
update_text4_df['int_grad'] = text4_df['int_grad']
update_text4_df['mlm_int_grad'] = text4_df['mlm_int_grad']
update_text4_df.to_csv('/content/drive/My Drive/LIN371/example_sent/text4_df.csv')



,token,simple_l2,mlm_simple_l2,int_grad,mlm_int_grad
0,[CLS],4.838400,0.944699,-0.143010,-0.058539
1,come,11.445228,1.945301,0.300826,0.393768
2,here,8.463614,1.428601,0.118139,-0.060454
3,you,6.194109,0.956207,0.300673,0.608146
4,cu,11.141582,1.585358,0.015734,-0.040207
5,##dd,16.401936,2.463371,0.602192,0.162703
6,##ly,6.853973,1.311297,0.078094,0.114794
7,little,8.996356,1.754293,0.091587,0.535713
8,ras,19.296671,3.632888,-0.096479,-0.229616
9,##cal,20.341785,3.129975,-0.166187,0.080376


In [ ]:
# from captum.attr import visualization as viz

# # storing couple samples in an array for visualization purposes
# start_position_vis = viz.VisualizationDataRecord(
#                         summarized_attributions[2],
#                         torch.max(torch.softmax(pred[2], dim=0)),
#                         torch.argmax(pred[2]),
#                         target_class[2],
#                         str(target_class[2]),
#                         summarized_attributions[2].sum(),
#                         tokenizer.convert_ids_to_tokens(input_ids[2].tolist()),
#                         delta)

# print('\033[1m', 'Visualizations For Start Position', '\033[0m')
# viz.visualize_text([start_position_vis])

 Visualizations For Start Position 


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,0 (0.94),tensor(1),0.30,[CLS] i could stare at those bo ##inger ##s for hours . [SEP]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,0 (0.94),tensor(1),0.30,[CLS] i could stare at those bo ##inger ##s for hours . [SEP]
